# Identifying Hard Coded Single Precision Variables

In looking to merge MARCS with DSEP, it is immediately clear that the two codes are incompatible when it comes to passing variables. MARCS is written with single precision declarations for real variables and DSEP is written with double precision declarations. Type conversions can be carried out in several different ways, but perhaps it is worth upgrading MARCS to double precision. However, while it is simple enough to redefine real declarations to `real(dp)`, where `dp` is a double precision type declaration, there are many instances of hard coded single precision variables. That is, variables defined with `X.XeYYY` instead of `X.XdYYY`. Finding such declarations is non-trivial.

In [1]:
import fileinput as fi

An initial attempt to extract all instances of hard coded single precision real variables was done using `grep` and a regular expression,
```bash
grep -i "[0-9].e.[0-9]" *.f > marcs_single_precision.txt
```
where `-i` flags the result to be case insensitive. Let's read in the result.

In [2]:
single_precision_vars = [line.rstrip('\n') for line in fi.input('marcs_single_precision.txt')]

Take a quick look at the first few lines,

In [3]:
print single_precision_vars[:5]

['CIAh2h.f:75:        propac=1.e-30', 'CIAh2h2.f:75:        propac=1.e-30', 'CIAh2he.f:75:        propac=1.e-30', 'CIAhhe.f:75:        propac=1.e-30', 'archiv.f:156:cc        print 215, k,log10(max(1.e-30,xmettryck(k,1))),']


Everything looks reasonble, but I know for a fact that scientific notation can also be declared in `FORMAT` statements. Therefore, it's necessary to remove these from the entry list.

In [4]:
single_precision_vars = [line for line in single_precision_vars if line.lower().find('format') == -1]

Make sure we haven't removed valid entries, as compared to the initial output.

In [5]:
print single_precision_vars[:5]

['CIAh2h.f:75:        propac=1.e-30', 'CIAh2h2.f:75:        propac=1.e-30', 'CIAh2he.f:75:        propac=1.e-30', 'CIAhhe.f:75:        propac=1.e-30', 'archiv.f:156:cc        print 215, k,log10(max(1.e-30,xmettryck(k,1))),']


We can now organize the data by subroutine and output the lines on which a single precision real is expcted to occur. There will be false positives, but those can be checked by eye.

In [6]:
file_stream = open('single_precision_index.txt', 'w')
routine = ''
for line in single_precision_vars:
    line = line.split(':')
    if line[0] == routine:
        file_stream.write('\t{:4s} :: {:50s}\n'.format(line[1].strip(), line[2].strip()))
    else:
        routine = line[0]
        file_stream.write('\n\n{:s}\n\n'.format(line[0].rstrip('.f').upper()))
        file_stream.write('\t{:4s} :: {:70s}\n'.format(line[1].strip(), line[2].strip()))
file_stream.close()

Markdown is also a helpful format.

In [7]:
file_stream = open('single_precision_index.md', 'w')
routine = ''
for line in single_precision_vars:
    line = line.split(':')
    if line[0] == routine:
        file_stream.write('\t{:4s} :: {:50s}\n'.format(line[1].strip(), line[2].strip()))
    else:
        routine = line[0]
        file_stream.write('\n\n# {:s}\n\n'.format(line[0].rstrip('.f').upper()))
        file_stream.write('\t{:4s} :: {:70s}\n'.format(line[1].strip(), line[2].strip()))
file_stream.close()